In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification, pipeline
from datasets import load_dataset, Features, Sequence, Value, Dataset
import torch
import tqdm

In [2]:
tmp = Dataset.from_pandas(pd.read_csv('train.csv').fillna('').head(100))
data = tmp.map(lambda ex: {'labels': ex['target']}, batched=True).remove_columns(['target', 'standard_error', 'id'])

  0%|          | 0/1 [00:00<?, ?ba/s]

In [3]:
MODEL_NAME = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=1)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [4]:
MAX_LENGTH = 512
data = data.map(lambda e: tokenizer(e['excerpt'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)
data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'labels'])

  0%|          | 0/1 [00:00<?, ?ba/s]

In [9]:
dataloader = torch.utils.data.DataLoader(data, batch_size=32)

In [10]:
class AttentionHead(torch.nn.Module):
    def __init__(self, in_features, hidden_dim, num_targets):
        super().__init__()
        self.in_features = in_features
        self.middle_features = hidden_dim

        self.W = torch.nn.Linear(in_features, hidden_dim)
        self.V = torch.nn.Linear(hidden_dim, 1)
        self.out_features = hidden_dim

    def forward(self, features):
        att = torch.tanh(self.W(features))

        score = self.V(att)

        attention_weights = torch.softmax(score, dim=1)

        context_vector = attention_weights * features
        context_vector = torch.sum(context_vector, dim=1)

        return context_vector

In [17]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.head = AttentionHead(768,768,1)
        self.dropout = torch.nn.Dropout(0.1)
        self.linear = torch.nn.Linear(self.head.out_features,1)

    def forward(self,**xb):
        x = self.head(xb[0])
        return x

In [19]:
model = Model()
model = model.from_pretrained('roberta-base')
model.eval()

AttributeError: 'Model' object has no attribute 'from_pretrained'

In [15]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_backward_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_version',
 'add_module',
 'apply',


In [13]:


embeddings = list()
with torch.no_grad():
    for i, inputs in tqdm(enumerate(dataloader)):
        inputs = {key:val.reshape(val.shape[0],-1).to(device) for key,val in inputs.items()}
        outputs = model(**inputs)
        outputs = outputs.detach().cpu().numpy()
        embeddings.extend(outputs)

TypeError: 'module' object is not callable

In [5]:
# Compute metrics is an argument of the Trainer
def compute_metrics(pred_results):
    """For computing RMSE inside the training loop"""
    y_pred = pred_results.predictions.squeeze()
    y_true = pred_results.label_ids
    return {"rmse": rmse(y_true, y_pred)}

def rmse(y_true, y_pred):
    return np.sqrt(((y_true - y_pred) ** 2).mean().item())